In [ ]:
import boto3, botocore
from botocore.exceptions import ClientError
import os, time, json, io, zipfile
from datetime import date
from dotenv import load_dotenv


from misc import load_from_yaml, save_to_yaml
import iam, s3, lf, rds, vpc, ec2

load_dotenv(".env")
# boto3.setup_default_session(profile_name="AMominNJ")

True

In [ ]:
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ['AWS_DEFAULT_REGION']
VPC_ID            = os.environ['AWS_DEFAULT_VPC']
SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
SUBNET_ID         = SUBNET_IDS[0]
print(SUBNET_IDS)

In [ ]:
ec2_client           = boto3.client('ec2', region_name=REGION)
ec2_resource         = boto3.resource('ec2', region_name=REGION)
sts_client           = boto3.client('sts')
rds_client           = boto3.client('rds')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
stepfunctions_client = boto3.client('stepfunctions')
apigateway_client    = boto3.client('apigateway')
lsn_client           = boto3.client('lambda')
events_client        = boto3.client('events')

sqs_client           = boto3.client('sqs')

-   [Getting Started With AWS Cloud | Step-by-Step Guide](https://www.youtube.com/watch?v=CjKhQoYeR4Q&t=29s)

#### `Client` vs `Resource`

In boto3, `client` and `resource` are two different ways to interact with AWS services. Both provide access to AWS services but differ in their interface, functionality, and ease of use. Here's a breakdown of their differences:

-   **client**:

    -   Provides low-level access to AWS services.
    -   Requires you to call methods directly and handle the response as dictionaries.
    -   You must know AWS API operations and parameters in detail to use client effectively.
    -   It is essentially a one-to-one mapping with AWS API.

-   **resource**:

    -   Provides a higher-level, object-oriented abstraction for AWS services.
    -   You work with resources like Instance, Bucket, Table, etc., as Python objects rather than dealing with JSON responses.
    -   Easier to use for object-based interactions, as it abstracts away some of the details of the API.

# IAM

In [ ]:
# [method for method in dir(iam_client) if not method.startswith("_")]

['add_client_id_to_open_id_connect_provider',
 'add_role_to_instance_profile',
 'add_user_to_group',
 'attach_group_policy',
 'attach_role_policy',
 'attach_user_policy',
 'can_paginate',
 'change_password',
 'close',
 'create_access_key',
 'create_account_alias',
 'create_group',
 'create_instance_profile',
 'create_login_profile',
 'create_open_id_connect_provider',
 'create_policy',
 'create_policy_version',
 'create_role',
 'create_saml_provider',
 'create_service_linked_role',
 'create_service_specific_credential',
 'create_user',
 'create_virtual_mfa_device',
 'deactivate_mfa_device',
 'delete_access_key',
 'delete_account_alias',
 'delete_account_password_policy',
 'delete_group',
 'delete_group_policy',
 'delete_instance_profile',
 'delete_login_profile',
 'delete_open_id_connect_provider',
 'delete_policy',
 'delete_policy_version',
 'delete_role',
 'delete_role_permissions_boundary',
 'delete_role_policy',
 'delete_saml_provider',
 'delete_server_certificate',
 'delete_service_linked_role',
 'delete_service_specific_credential',
 'delete_signing_certificate',
 'delete_ssh_public_key',
 'delete_user',
 'delete_user_permissions_boundary',
 'delete_user_policy',
 'delete_virtual_mfa_device',
 'detach_group_policy',
 'detach_role_policy',
 'detach_user_policy',
 'enable_mfa_device',
 'exceptions',
 'generate_credential_report',
 'generate_organizations_access_report',
 'generate_presigned_url',
 'generate_service_last_accessed_details',
 'get_access_key_last_used',
 'get_account_authorization_details',
 'get_account_password_policy',
 'get_account_summary',
 'get_context_keys_for_custom_policy',
 'get_context_keys_for_principal_policy',
 'get_credential_report',
 'get_group',
 'get_group_policy',
 'get_instance_profile',
 'get_login_profile',
 'get_mfa_device',
 'get_open_id_connect_provider',
 'get_organizations_access_report',
 'get_paginator',
 'get_policy',
 'get_policy_version',
 'get_role',
 'get_role_policy',
 'get_saml_provider',
 'get_server_certificate',
 'get_service_last_accessed_details',
 'get_service_last_accessed_details_with_entities',
 'get_service_linked_role_deletion_status',
 'get_ssh_public_key',
 'get_user',
 'get_user_policy',
 'get_waiter',
 'list_access_keys',
 'list_account_aliases',
 'list_attached_group_policies',
 'list_attached_role_policies',
 'list_attached_user_policies',
 'list_entities_for_policy',
 'list_group_policies',
 'list_groups',
 'list_groups_for_user',
 'list_instance_profile_tags',
 'list_instance_profiles',
 'list_instance_profiles_for_role',
 'list_mfa_device_tags',
 'list_mfa_devices',
 'list_open_id_connect_provider_tags',
 'list_open_id_connect_providers',
 'list_policies',
 'list_policies_granting_service_access',
 'list_policy_tags',
 'list_policy_versions',
 'list_role_policies',
 'list_role_tags',
 'list_roles',
 'list_saml_provider_tags',
 'list_saml_providers',
 'list_server_certificate_tags',
 'list_server_certificates',
 'list_service_specific_credentials',
 'list_signing_certificates',
 'list_ssh_public_keys',
 'list_user_policies',
 'list_user_tags',
 'list_users',
 'list_virtual_mfa_devices',
 'meta',
 'put_group_policy',
 'put_role_permissions_boundary',
 'put_role_policy',
 'put_user_permissions_boundary',
 'put_user_policy',
 'remove_client_id_from_open_id_connect_provider',
 'remove_role_from_instance_profile',
 'remove_user_from_group',
 'reset_service_specific_credential',
 'resync_mfa_device',
 'set_default_policy_version',
 'set_security_token_service_preferences',
 'simulate_custom_policy',
 'simulate_principal_policy',
 'tag_instance_profile',
 'tag_mfa_device',
 'tag_open_id_connect_provider',
 'tag_policy',
 'tag_role',
 'tag_saml_provider',
 'tag_server_certificate',
 'tag_user',
 'untag_instance_profile',
 'untag_mfa_device',
 'untag_open_id_connect_provider',
 'untag_policy',
 'untag_role',
 'untag_saml_provider',
 'untag_server_certificate',
 'untag_user',
 'update_access_key',
 'update_account_password_policy',
 'update_assume_role_policy',
 'update_group',
 'update_login_profile',
 'update_open_id_connect_provider_thumbprint',
 'update_role',
 'update_role_description',
 'update_saml_provider',
 'update_server_certificate',
 'update_service_specific_credential',
 'update_signing_certificate',
 'update_ssh_public_key',
 'update_user',
 'upload_server_certificate',
 'upload_signing_certificate',
 'upload_ssh_public_key',
 'waiter_names']

In [ ]:
def list_iam_roles():
    roles = []
    paginator = iam_client.get_paginator('list_roles')
    for page in paginator.paginate():
        roles.extend(page['Roles'])
    return roles

# List all roles
roles = list_iam_roles()

# Print the role names
for role in roles:
    print(f"Role Name: {role['RoleName']}, ARN: {role['Arn']}")


In [11]:
def list_instance_profiles():
    """Lists all instance profiles in the current AWS account."""


    response = iam_client.list_instance_profiles()
    instance_profiles = response['InstanceProfiles']

    while 'Marker' in response:
        response = iam_client.list_instance_profiles(Marker=response['Marker'])
        instance_profiles.extend(response['InstanceProfiles'])

    for profile in instance_profiles:
        print(profile['InstanceProfileName'])

In [12]:
list_instance_profiles()

EMR_EC2_DefaultRole


In [ ]:
# Replace with the instance ID and instance profile name
instance_id = 'i-xxxxxxxxxxxxxxxxx'
instance_profile_name = 'InstanceProfileName'

response = ec2_client.disassociate_iam_instance_profile(
    AssociationId=instance_id,
)
print("Instance profile detached:", response)


In [ ]:
# Replace with your instance profile and role names
instance_profile_name = 'MyInstanceProfile'
role_name = 'MyRole'

response = iam_client.remove_role_from_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName=role_name
)
print("Role removed from instance profile:", response)


In [ ]:
response = iam_client.delete_instance_profile(
    InstanceProfileName=instance_profile_name
)
print("Instance profile deleted:", response)


# [S3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html)

1. s3_client`.close( ... )`
1. s3_client`.copy( ... )`
1. s3_client`.copy_object( ... )`
1. s3_client`.create_bucket( ... )`
1. s3_client`.create_session( ... )`
1. s3_client`.delete_bucket( ... )`
1. s3_client`.delete_bucket_cors( ... )`
1. s3_client`.delete_bucket_encryption( ... )`
1. s3_client`.delete_bucket_metrics_configuration( ... )`
1. s3_client`.delete_bucket_policy( ... )`
1. s3_client`.delete_object( ... )`
1. s3_client`.delete_objects( ... )`
1. s3_client`.download_file( ... )`
1. s3_client`.download_fileobj( ... )`
1. s3_client`.get_bucket_acl( ... )`
1. s3_client`.get_bucket_cors( ... )`
1. s3_client`.get_bucket_encryption( ... )`
1. s3_client`.get_bucket_location( ... )`
1. s3_client`.get_bucket_logging( ... )`
1. s3_client`.get_bucket_metrics_configuration( ... )`
1. s3_client`.get_bucket_notification( ... )`
1. s3_client`.get_bucket_notification_configuration( ... )`
1. s3_client`.get_bucket_policy( ... )`
1. s3_client`.get_bucket_policy_status( ... )`
1. s3_client`.get_bucket_request_payment( ... )`
1. s3_client`.get_bucket_versioning( ... )`
1. s3_client`.get_object( ... )`
1. s3_client`.get_object_acl( ... )`
1. s3_client`.get_object_attributes( ... )`
1. s3_client`.get_public_access_block( ... )`
1. s3_client`.head_bucket( ... )`
1. s3_client`.head_object( ... )`
1. s3_client`.list_bucket_metrics_configurations( ... )`
1. s3_client`.list_buckets( ... )`
1. s3_client`.list_directory_buckets( ... )`
1. s3_client`.list_object_versions( ... )`
1. s3_client`.list_objects( ... )`
1. s3_client`.list_objects_v2( ... )`
1. s3_client`.put_bucket_acl( ... )`
1. s3_client`.put_bucket_cors( ... )`
1. s3_client`.put_bucket_encryption( ... )`
1. s3_client`.put_bucket_logging( ... )`
1. s3_client`.put_bucket_metrics_configuration( ... )`
1. s3_client`.put_bucket_notification( ... )`
1. s3_client`.put_bucket_notification_configuration( ... )`
1. s3_client`.put_bucket_policy( ... )`
1. s3_client`.put_bucket_request_payment( ... )`
1. s3_client`.put_bucket_tagging( ... )`
1. s3_client`.put_bucket_versioning( ... )`
1. s3_client`.put_object( ... )`
1. s3_client`.put_object_acl( ... )`
1. s3_client`.put_object_retention( ... )`
1. s3_client`.restore_object( ... )`
1. s3_client`.select_object_content( ... )`
1. s3_client`.upload_file( ... )`
1. s3_client`.upload_fileobj( ... )`

In [ ]:
# [method for method in dir(s3_client) if not method.startswith("_")]

-   **Create a Bucket**: Creates a new S3 bucket.

In [ ]:
S3_BUCKET_DATALAKE = "httx-datalake-bkt"
S3_BUCKET_GLUE_ASSETS = "httx-glue-assets-bkt"

In [ ]:
s3 = boto3.resource('s3')
bucket1 = s3.Bucket(S3_BUCKET_DATALAKE)
bucket2 = s3.Bucket(S3_BUCKET_GLUE_ASSETS)

# Delete all objects in the bucket
bucket1.objects.all().delete()
bucket2.objects.all().delete()

# Delete all object versions (if versioning is enabled)
# bucket1.object_versions.all().delete()
# bucket2.object_versions.all().delete()

# Finally, delete the bucket
bucket1.delete()
bucket2.delete()

In [ ]:
acl = 'public-read'                         # Set the ACL (e.g., 'private', 'public-read')
enable_versioning = False                   # Enable versioning
enable_encryption = False                   # Enable server-side encryption

folders1 = ['raw/customers/', 'cleansed/customers/']
folders2 = ['temporary/', 'sparkHistoryLogs/']

In [ ]:
s3_client.create_bucket(Bucket=S3_BUCKET_DATALAKE)
s3_client.create_bucket(Bucket=S3_BUCKET_GLUE_ASSETS)

-   **Put Object (Upload)**: Uploads an object directly to S3 (binary or text content).

In [ ]:
# s3_client.put_object(Bucket='my-bucket', Key='new_file.txt', Body=b'Hello, World!')

In [ ]:
[s3_client.put_object(Bucket=S3_BUCKET_DATALAKE, Key=folder) for folder in folders1]
[s3_client.put_object(Bucket=S3_BUCKET_GLUE_ASSETS, Key=folder) for folder in folders2]

-   **List All Buckets**: Lists all the buckets in your S3 account.

In [ ]:
response = s3_client.list_buckets()
print(response)
for bucket in response['Buckets']:
    print(f'Bucket: {bucket["Name"]}')

-   **Upload a File to S3**: Uploads a file to a specified S3 bucket.

In [ ]:
s3_client.upload_file('./data/customers.csv', S3_BUCKET_DATALAKE, 'raw/customers/customers.csv')

-   **Download a File from S3**: Downloads a file from an S3 bucket to a local directory.

In [ ]:
s3_client.download_file(S3_BUCKET_DATALAKE, 'raw/customers/customers.csv', './data/downloaded_file.csv')

-   **List Objects in a Bucket**: Lists all objects within a bucket.

In [ ]:
response = s3_client.list_objects_v2(Bucket=S3_BUCKET_DATALAKE, Prefix="r")
print(response)
for obj in response.get('Contents', []):
    print(f'Object: {obj["Key"]}')

In [ ]:
# Check if the response contains any contents
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print(f"No objects found with prefix 'r' in bucket '{S3_BUCKET_DATALAKE}'.")


-   **Get Object (Download)**: Retrieves an object directly from S3.

In [ ]:
response = s3_client.get_object(Bucket=S3_BUCKET_DATALAKE, Key='raw/customers/customers.csv')
print(response)
content = response['Body'].read()
print(content.decode('utf-8'))

-   **Copy an Object to Another Bucket**: Copies an object from one bucket to another.

In [ ]:
copy_source = {'Bucket': S3_BUCKET_DATALAKE, 'Key': 'raw/customers/customers.csv'}
s3_client.copy_object(CopySource=copy_source, Bucket=S3_BUCKET_GLUE_ASSETS, Key='copied-file.txt')

-   **Delete an Object**: Deletes an object from a bucket.

In [ ]:
s3_client.delete_object(Bucket=S3_BUCKET_GLUE_ASSETS, Key='copied-file.txt')

- **Put Configuration**: 

In [ ]:
# The configuration 'EventBridgeConfiguration': {} to route all S3 events (like s3:ObjectCreated:* events) to EventBridge.

# Define the bucket notification configuration for EventBridge
notification_configuration = {
    'EventBridgeConfiguration': {}
}

# Put bucket notification configuration
response = s3_client.put_bucket_notification_configuration(
    Bucket=bucket_name,
    NotificationConfiguration=notification_configuration
)

print("Bucket notification configuration response:")
print(response)

In [ ]:
# Add S3 trigger to the Lambda function
response = s3_client.put_bucket_notification_configuration(
    Bucket=S3_BUCKET_DATALAKE,
    NotificationConfiguration={
        'LambdaFunctionConfigurations': [
            {
                'LambdaFunctionArn': LFN_CRAWLER_ARN,
                'Events': [
                    's3:ObjectCreated:*'  # Trigger Lambda on object creation
                ],
                'Filter': {
                    'Key': {
                        'FilterRules': [
                            {
                                'Name': 'prefix',
                                'Value': 'raw/customers/'  # Trigger only on this prefix
                            },
                        ]
                    }
                }
            }
        ]
    }
)

-   **Generate a Pre-signed URL**: Generates a pre-signed URL for secure access to a file.

In [ ]:
url = s3_client.generate_presigned_url('get_object',
                                Params={'Bucket': 'my-bucket', 'Key': 'uploaded_file.txt'},
                                ExpiresIn=3600)
print(f"Presigned URL: {url}")

-   **Check if an Object Exists**: Retrieves metadata from an object without returning the object itself, useful for checking if a file exists.

In [ ]:
try:
    s3_client.head_object(Bucket='my-bucket', Key='file.txt')
    print("Object exists")

except ClientError:
    print("Object does not exist")


-   **Delete a Bucket**: Deletes an empty bucket.

In [ ]:
s3_client.delete_bucket(Bucket='my-bucket')

- **Creating a Public S3 Bucket**

In [ ]:
# Initialize boto3 S3 client
s3 = boto3.client('s3')

# Replace with your desired bucket name and region
bucket_name = "your-public-bucket-name"
region = "us-east-1"  # Specify your AWS region

# Step 1: Create the S3 bucket
if region == "us-east-1":
    response = s3.create_bucket(Bucket=bucket_name)
else:
    response = s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={"LocationConstraint": region},
    )
print(f"Bucket '{bucket_name}' created successfully.")

In [ ]:

# Step 2: Set the bucket to be publicly accessible using a bucket policy
bucket_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadGetObject",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": f"arn:aws:s3:::{S3_BUCKET_DATALAKE}/*",
        }
    ],
}

# Convert the policy to a JSON string
bucket_policy_json = json.dumps(bucket_policy)

# Apply the bucket policy
s3_client.put_bucket_policy(Bucket=S3_BUCKET_DATALAKE, Policy=bucket_policy_json)
print(f"Public read access policy applied to bucket '{S3_BUCKET_DATALAKE}'.")

In [ ]:
# Step 3: Enable bucket ACL for public read access (optional, not needed if bucket policy is applied)
s3_client.put_bucket_acl(Bucket=S3_BUCKET_GLUE_ASSETS, ACL="public-read")
print(f"Bucket ACL set to 'public-read' for '{S3_BUCKET_GLUE_ASSETS}'.")

print("Public S3 bucket setup completed successfully!")

In [ ]:
import boto3
from botocore.config import Config
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

# Configure S3 client for anonymous access
try:
    public_s3_client = boto3.client(
        's3',
        config=Config(signature_version='s3v4'),
        aws_access_key_id=None,
        aws_secret_access_key=None,
    )

    # Step 1: List objects in the public bucket
    print(f"Listing objects in the bucket '{S3_BUCKET_GLUE_ASSETS}':")
    response = public_s3_client.list_objects_v2(Bucket=S3_BUCKET_GLUE_ASSETS)
    for obj in response.get('Contents', []):
        print(f"- {obj['Key']}")
    
    response = s3_client.get_object(Bucket=S3_BUCKET_GLUE_ASSETS, Key='copied-file.txt')
    print(response)
    content = response['Body'].read()
    print(content.decode('utf-8'))


except NoCredentialsError:
    print("Error: No credentials provided. Anonymous access failed.")
except PartialCredentialsError:
    print("Error: Partial credentials provided. Anonymous access failed.")
except Exception as e:
    print(f"An error occurred: {e}")


# [CloudWatch](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/cloudwatch.html)

- [CloudWatchLogs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/logs.html)

In [ ]:
# Create a CloudWatch client for Metrics
cw_metrics_client = boto3.client('cloudwatch')

# Create a CloudWatch client for Logs
logs_client = boto3.client('logs')

# Create a CloudWatch client for Events (EventBridge)
cw_events_client = boto3.client('events')

# Create a CloudWatch client for Synthetics
cw_synthetics_client = boto3.client('synthetics')

In [ ]:
logs_client = boto3.client('logs')

# Specify the log group name (typically prefixed with /aws/lambda/)
log_group_name = '/aws/lambda/lambda-function-name'

# Create the CloudWatch log group
response = logs_client.create_log_group(logGroupName=log_group_name)

In [ ]:
# Define the log group name and log stream name
log_group_name = '/aws-glue/crawlers'
log_stream_name = 'httx-s3crawler'

# Fetch logs
response = logs_client.get_log_events(
    logGroupName=log_group_name,
    logStreamName=log_stream_name,
    startFromHead=False,  # Set to False if you want the latest logs first
    limit = 10
)

# Print the log events
for event in response['events']:
    print(event['message'])

In [ ]:
response = logs_client.filter_log_events(
    logGroupName='string',
    logGroupIdentifier='string',
    logStreamNames=[
        'string',
    ],
    logStreamNamePrefix='string',
    startTime=123,
    endTime=123,
    filterPattern='string',
    nextToken='string',
    limit=123,
    interleaved=True|False,
    unmask=True|False
)

In [ ]:
log_group_name = '/aws/lambda/example-log-group'

# Delete the log group:
logs_client.delete_log_group(logGroupName=log_group_name)

##### 1. **put_metric_data**: Sends metric data points to CloudWatch.


In [ ]:
# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

# Put custom metric data
response = cloudwatch.put_metric_data(
    Namespace='MyCustomNamespace',
    MetricData=[
        {
            'MetricName': 'MyCustomMetric',
            'Value': 100.0,
            'Unit': 'None',
        },
    ]
)
print("Put Metric Data Response:", response)


In [ ]:
# List metrics for AWS Lambda
lambda_metrics = cloudwatch.list_metrics(
    Namespace='AWS/Lambda',
    MetricName='Invocations',  # You can change this to other metric names
    Dimensions=[
        {
            'Name': 'FunctionName',
            'Value': 'HelloWorldFunction'  # Replace with your function name
        }
    ]
)

print("AWS Lambda Metrics for 'HelloWorldFunction':")
for metric in lambda_metrics['Metrics']:
    print(f"Metric Name: {metric['MetricName']}, Dimensions: {metric['Dimensions']}")


##### 2. **get_metric_data**: Retrieves metric data from CloudWatch over a specified time period.

In [ ]:
from datetime import datetime, timedelta

# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

# Get metric data
end_time = datetime.utcnow()
start_time = end_time - timedelta(hours=1)

response = cloudwatch.get_metric_data(
    MetricDataQueries=[
        {
            'Id': 'm1',
            'MetricStat': {
                'MetricName': 'MyCustomMetric',
                'Namespace': 'MyCustomNamespace',
                'Stat': 'Average',
                'Period': 300,
            },
            'ReturnData': True,
        },
    ],
    StartTime=start_time,
    EndTime=end_time,
)

print("Get Metric Data Response:", response)


In [ ]:
# Specify the function name
function_name = 'HelloWorldFunction'  # Replace with your Lambda function name

# Define the time range for the metrics
end_time = datetime.utcnow()  # Current time
start_time = end_time - timedelta(hours=1)  # One hour ago

# Define the metric data queries
metric_data_queries = [
    {
        'Id': 'invocations',
        'MetricStat': {
            'MetricName': 'Invocations',
            'Namespace': 'AWS/Lambda',
            'Stat': 'Sum',  # You can also use Average, Maximum, etc.
            'Period': 60,  # Data points are aggregated over 1-minute intervals
        },
        'ReturnData': True,
    },
    {
        'Id': 'errors',
        'MetricStat': {
            'MetricName': 'Errors',
            'Namespace': 'AWS/Lambda',
            'Stat': 'Sum',
            'Period': 60,
        },
        'ReturnData': True,
    },
    {
        'Id': 'duration',
        'MetricStat': {
            'MetricName': 'Duration',
            'Namespace': 'AWS/Lambda',
            'Stat': 'Average',
            'Period': 60,
        },
        'ReturnData': True,
    },
]

# Retrieve metric data
response = cloudwatch.get_metric_data(
    MetricDataQueries=metric_data_queries,
    StartTime=start_time,
    EndTime=end_time,
)

# Print the retrieved metrics
print("AWS Lambda Metrics Data:")
for result in response['MetricDataResults']:
    print(f"Metric: {result['Label']}")
    for timestamp, value in zip(result['Timestamps'], result['Values']):
        print(f"Time: {timestamp}, Value: {value}")


##### 3. **get_metric_statistics**: Retrieves historical statistics for a specified metric.

In [ ]:
from datetime import datetime, timedelta

# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

# Get metric statistics
end_time = datetime.utcnow()
start_time = end_time - timedelta(days=1)

response = cloudwatch.get_metric_statistics(
    Namespace='MyCustomNamespace',
    MetricName='MyCustomMetric',
    StartTime=start_time,
    EndTime=end_time,
    Period=3600,
    Statistics=['Average', 'Sum'],
)

print("Get Metric Statistics Response:", response)




##### 4. **list_metrics**: Lists the specified metrics.


In [ ]:

# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

# List metrics for AWS Lambda
lambda_metrics = cloudwatch.list_metrics(
    Namespace='AWS/Lambda'  # The namespace for AWS Lambda metrics
)

print("AWS Lambda Metrics:")
for metric in lambda_metrics['Metrics']:
    print(f"Metric Name: {metric['MetricName']}, Dimensions: {metric['Dimensions']}")


In [ ]:
# List metrics for AWS EC2
ec2_metrics = cloudwatch.list_metrics(
    Namespace='AWS/EC2'  # The namespace for AWS EC2 metrics
)

print("\nAWS EC2 Metrics:")
for metric in ec2_metrics['Metrics']:
    print(f"Metric Name: {metric['MetricName']}, Dimensions: {metric['Dimensions']}")

##### 5. **create_alarm**: Creates a CloudWatch alarm based on a specified metric.

In [ ]:
# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

# Create an alarm
response = cloudwatch.create_alarm(
    AlarmName='MyCustomAlarm',
    MetricName='MyCustomMetric',
    Namespace='MyCustomNamespace',
    Statistic='Average',
    Period=300,
    Threshold=50.0,
    ComparisonOperator='GreaterThanThreshold',
    ActionsEnabled=True,
    AlarmActions=[
        'arn:aws:sns:us-east-1:123456789012:MySNSTopic',
    ],
)

print("Create Alarm Response:", response)


##### 6. **describe_alarms**: Describes one or more CloudWatch alarms.

In [ ]:

# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

# Describe alarms
response = cloudwatch.describe_alarms()

print("Describe Alarms Response:")
for alarm in response['MetricAlarms']:
    print(alarm)


##### 7. **delete_alarms**: Deletes one or more specified alarms.

In [ ]:

# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

# Delete alarms
response = cloudwatch.delete_alarms(
    AlarmNames=['MyCustomAlarm']
)

print("Delete Alarms Response:", response)


##### 8. **set_alarm_state**: Sets the state of an alarm.

In [ ]:

# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

# Set alarm state
response = cloudwatch.set_alarm_state(
    AlarmName='MyCustomAlarm',
    StateValue='ALARM',
    StateReason='Setting the alarm state for testing purposes.'
)

print("Set Alarm State Response:", response)


##### 9. **put_dashboard**: Creates or updates a CloudWatch dashboard.

In [ ]:

# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

# Put dashboard
response = cloudwatch.put_dashboard(
    DashboardName='MyDashboard',
    DashboardBody='{"widgets":[{"type":"metric","x":0,"y":0,"width":6,"height":6,"properties":{"metrics":[["MyCustomNamespace","MyCustomMetric"]]}}]}'
)

print("Put Dashboard Response:", response)


##### 10. **list_dashboards**: Lists all CloudWatch dashboards in the account.

In [ ]:

# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

# List dashboards
response = cloudwatch.list_dashboards()

print("List Dashboards Response:")
for dashboard in response['DashboardEntries']:
    print(dashboard)


# [EventBridge](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/events.html)

- [Amazon EventBridge Pipes - A Crash Course](https://www.youtube.com/watch?v=TE6tnOegt7A)
- [Why wasn't my Lambda function triggered by my EventBridge rule?](https://www.youtube.com/watch?v=95F3Fam4h3E)
- [Solving the DynamoDB EventBridge Pipes Problem Jason Wadsworth](https://www.youtube.com/watch?v=W5pMBMhUnic)

##### Event Rule-1

In [ ]:
S3_RAW_CRAWLER_RULE_NAME1 = 'httx-raw-crawler-rule'
JOB_COMPLETE_RULE_NAME = 'httx-job-complete-rule'
JOB_COMPLETE_TOPIC_ARN = ""

In [ ]:
crawler_rule_event_pattern = {
  "source": ["aws.glue"],
  "detail-type": ["Glue Crawler State Change"],
  "detail": {
    "state": ["Succeeded"],
    "crawlerName": ['CRAWLER_NAME']
  }
}

# Create EventBridge Rule to catch Glue Crawler State Change events
S3_RAW_CRAWLER_RULE_ARN = events_client.put_rule(
    Name=S3_RAW_CRAWLER_RULE_NAME1,
    EventPattern=json.dumps(crawler_rule_event_pattern),
    State='ENABLED',
    Description='Rule to capture AWS Glue Crawler state changes',
)['RuleArn']

##### Event Rule-2

In [ ]:
job_rule_event_pattern = {
    "source": ["aws.glue"],
    "detail-type": ["Glue Job State Change"], # Event Type
    # "detail": {
    #     "JobName": [JOB_NAME],
    #     "state": ["SUCCEEDED"]
    # }
}

response = events_client.put_rule(
    Name=JOB_COMPLETE_RULE_NAME,
    EventPattern=json.dumps(job_rule_event_pattern),
    State='ENABLED',
    Description='Rule to capture AWS Glue job state changes',
)

# Attach the Lambda function as a target to the EventBridge Rule
response = events_client.put_targets(
    Rule=JOB_COMPLETE_RULE_NAME,
    Targets=[{
        'Id': f"{JOB_COMPLETE_RULE_NAME}_target",
        'Arn': JOB_COMPLETE_TOPIC_ARN
    }]
)

In [ ]:
# !aws events list-rules --name-prefix {JOB_COMPLETE_RULE_NAME}

In [ ]:
# List all rules associated with the given prefix
rules = events_client.list_rules(NamePrefix="httx")['Rules']

# List all targates associated with each rule
targets_list = [events_client.list_targets_by_rule(Rule=rule['Name'])['Targets'] for rule in rules]

# Remove all targets associated with each rule
[events_client.remove_targets(Rule=rule['Name'], Ids=[target['Id'] for target in targets]) for rule, targets, in zip(rules, targets_list)]

# Delete all rules
[events_client.delete_rule(Name=rule['Name']) for rule in rules]

In [ ]:

# events_client.list_targets_by_rule(Rule=JOB_COMPLETE_RULE_NAME)['Targets']
# events_client.remove_targets(Rule=JOB_COMPLETE_RULE_NAME, Ids=[targets[0]['Id']])

# events_client.list_rules(NamePrefix=JOB_COMPLETE_RULE_NAME)
# events_client.delete_rule(Name=JOB_COMPLETE_RULE_NAME)


##### Test the Rule with dummy event

In [ ]:
event_pattern = {
    'Source': ['my.custom.source'],  # Custom source for testing
    'DetailType': ['Glue Job State Change'],
    'Detail': {
        'state': ['SUCCEEDED']
    }
}

test_event = {
    'Source': 'my.custom.source',
    'DetailType': 'Glue Job State Change',
    'Detail': json.dumps({
        'state': 'SUCCEEDED'              # This should match the state you're watching for
    })
}

response = events_client.put_events(
    Entries=[
        test_event
    ]
)

print(response)

# [Step Function](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/stepfunctions.html)

In [ ]:
# 3. Create Step Functions state machine
state_machine_definition = {
    "Comment": "Store Checkout Flow",
    "StartAt": "checkInventory",
    "States": {
        "checkInventory": {
            "Type": "Task",
            "Resource": lambda_arns['checkInventory'],
            "Catch": [
                {
                    "ErrorEquals": ["BookNotFound"],
                    "Next": "BookNotFoundError"
                },
                {
                    "ErrorEquals": ["BookOutOfStock"],
                    "Next": "BookOutOfStockError"
                }
            ],
            "ResultPath": "$.book",
            "Next": "calculateTotal"
        },
        "calculateTotal": {
            "Type": "Task",
            "Resource": lambda_arns['calculateTotal'],
            "ResultPath": "$.total",
            "Next": "isRedeemNeeded"
        },
        "isRedeemNeeded": {
            "Type": "Choice",
            "Choices": [
                {
                    "Variable": "$.redeem",
                    "BooleanEquals": True,
                    "Next": "RedeemPoints"
                }
            ],
            "Default": "BillCustomer"
        },
        "RedeemPoints": {
            "Type": "Task",
            "Resource": lambda_arns['redeemPoints'],
            "ResultPath": "$.total",
            "Catch": [
                {
                    "ErrorEquals": ["States.ALL"],
                    "Next": "RedeemPointsError"
                }
            ],
            "Next": "BillCustomer"
        },
        "BillCustomer": {
            "Type": "Task",
            "Resource": lambda_arns['billCustomer'],
            "ResultPath": "$.billingStatus",
            "Retry": [
                {
                    "ErrorEquals": ["States.ALL"],
                    "MaxAttempts": 0
                }
            ],
            "Catch": [
                {
                    "ErrorEquals": ["States.ALL"],
                    "ResultPath": "$.customerBilling",
                    "Next": "BillingError"
                }
            ],
            "Next": "PrepareOrder"
        },
        "PrepareOrder": {
            "Type": "Task",
            "Resource": "arn:aws:states:::sqs:sendMessage.waitForTaskToken",
            "Parameters": {
                "QueueUrl": f"https://sqs.{region}.amazonaws.com/{account_id}/OrdersQueue",
                "MessageBody": {
                    "Input.$": "$",
                    "Token.$": "$$.Task.Token"
                }
            },
            "ResultPath": "$.courierStatus",
            "Catch": [
                {
                    "ErrorEquals": ["NoCourierAvailable"],
                    "ResultPath": "$.courierError",
                    "Next": "RefundCustomer"
                }
            ],
            "Next": "DispatchOrder"
        },
        "DispatchOrder": {
            "Type": "Task",
            "Resource": "arn:aws:states:::sns:publish",
            "Parameters": {
                "TopicArn": f"arn:aws:sns:{region}:{account_id}:NotifyCourier",
                "Message.$": "$"
            },
            "Next": "Dispatched"
        },
        "Dispatched": {
            "Type": "Pass",
            "Result": "Your order will be dispatched in 24 hours",
            "End": True
        },
        "RestoreRedeemPoints": {
            "Type": "Task",
            "Resource": lambda_arns['restoreRedeemPoints'],
            "End": True
        },
        "RestoreQuantity": {
            "Type": "Task",
            "Resource": lambda_arns['restoreQuantity'],
            "ResultPath": "$.quantityRestoreStatus",
            "Next": "RestoreRedeemPoints"
        },
        "RefundCustomer": {
            "Type": "Pass",
            "Result": "Customer is refunded",
            "ResultPath": "$.refundStatus",
            "Next": "RestoreQuantity"
        },
        "BookNotFoundError": {
            "Type": "Pass",
            "Result": "No such book available",
            "End": True
        },
        "BookOutOfStockError": {
            "Type": "Pass",
            "Result": "Sorry, the book is out of stock",
            "End": True
        },
        "RedeemPointsError": {
            "Type": "Pass",
            "Result": "Error in redeeming points",
            "End": True
        },
        "BillingError": {
            "Type": "Pass",
            "Result": "Billing error",
            "ResultPath": "$.billingStatus",
            "Next": "RestoreRedeemPoints"
        }
    }
}

state_machine_arn = stepfunctions_client.create_state_machine(
    name="storeCheckoutFlow",
    definition=json.dumps(state_machine_definition),
    roleArn=step_function_role_arn
)['stateMachineArn']

# [SQS](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sqs.html)

-   [AWS SQS FIFO Queues Overview and Demonstration](https://www.youtube.com/watch?v=cl_5dGGeTmY)
-   [AWS SQS to Lambda Tutorial in NodeJS | Step by Step](https://www.youtube.com/watch?v=JJQrVBRzlPg&t=762s)
-   [AWS SQS + Lambda Setup Tutorial - Step by Step](https://www.youtube.com/watch?v=xyHLX1dUwuA)
-   [Lambda + SQS Users Should Know About This](https://www.youtube.com/watch?v=0707Py8Jyf0)

In [ ]:
# [method for method in dir(sqs_client) if not method.startswith("_")]

In [ ]:
# ? sqs_client.add_permission

1. sqs_client.`add_permission( ... )`
1. sqs_client.`cancel_message_move_task( ... )`
1. sqs_client.`change_message_visibility( ... )`
1. sqs_client.`change_message_visibility_batch( ... )`
1. sqs_client.`close( ... )`
1. sqs_client.`create_queue( ... )`
1. sqs_client.`delete_message( ... )`
1. sqs_client.`delete_message_batch( ... )`
1. sqs_client.`delete_queue( ... )`
1. sqs_client.`generate_presigned_url( ... )`
1. sqs_client.`get_queue_attributes( ... )`
1. sqs_client.`get_queue_url( ... )`
1. sqs_client.`get_waiter( ... )`
1. sqs_client.`list_dead_letter_source_queues( ... )`
1. sqs_client.`list_message_move_tasks( ... )`
1. sqs_client.`list_queues( ... )`
1. sqs_client.`purge_queue( ... )`
1. sqs_client.`receive_message( ... )`
1. sqs_client.`remove_permission( ... )`
1. sqs_client.`send_message( ... )`
1. sqs_client.`send_message_batch( ... )`
1. sqs_client.`set_queue_attributes( ... )`
1. sqs_client.`start_message_move_task( ... )`

1. **MaximumMessageSize**

   - `Description`: Specifies the maximum allowed size of a single message in the queue.
   - `Default`: 262,144 bytes (256 KB).
   - `Range`: 1,024 bytes (1 KB) to 262,144 bytes (256 KB).
   - `Usage`: Set this attribute to control the maximum size of messages sent to the queue. If a message exceeds this size, it will be rejected.
   - `Impact`: Lower values reduce costs by limiting message size but may limit the types of data that can be sent in a single message. Larger values may increase costs but allow for larger payloads.

2. **MessageRetentionPeriod**

   - `Description`: Determines how long SQS retains messages in the queue after they are sent, even if they are not processed.
   - `Default`: 4 days (345,600 seconds).
   - `Range`: 60 seconds (1 minute) to 1,209,600 seconds (14 days).
   - `Usage`: Set the retention period based on the time you expect to need to process messages. For example, setting it to 14 days can help in scenarios where messages may not be processed immediately.
   - `Impact`: Longer retention periods help ensure that no message data is lost, but they increase storage costs. Shorter periods reduce storage costs but may delete messages before they are processed if there’s a delay.

3. **VisibilityTimeout**

   - `Description`: Controls the duration (in seconds) that a message remains invisible to other consumers after it has been received.
   - `Default`: 30 seconds.
   - `Range`: 0 seconds to 43,200 seconds (12 hours).
   - `Usage`: The visibility timeout prevents other consumers from receiving and processing the same message within the set time. If a consumer fails to process a message within this timeout, the message becomes visible again for another consumer to receive.
   - `Impact`: Short timeouts may lead to duplicate processing if processing takes longer than the timeout. Longer timeouts ensure that consumers have sufficient time to process messages but may delay message availability if a consumer fails.

4. **DelaySeconds**

   - `Description`: Sets a delay for the delivery of all messages added to the queue.
   - `Default`: 0 seconds (no delay).
   - `Range`: 0 seconds to 900 seconds (15 minutes).
   - `Usage`: Use this delay to postpone message availability. For example, in workflows where processing should start after a certain time, setting a delay ensures no message is consumed before that time.
   - `Impact`: All messages sent to the queue will be delayed by the specified number of seconds. This can help control processing timing but might slow down urgent workflows.

5. **ContentBasedDeduplication**

   - `Description`: Automatically deduplicates messages based on content for FIFO queues. This feature is only applicable to FIFO queues.
   - `Values`: `true` or `false`.
   - `Usage`: When enabled (`true`), SQS generates a unique `deduplication_id` based on the message content, which prevents duplicate messages within a 5-minute deduplication window.
   - `Impact`: Useful for ensuring idempotency in applications where duplicate messages can lead to redundant or conflicting actions. If disabled, deduplication depends on the `MessageDeduplicationId` parameter, which requires explicit configuration by the message sender.

6. **RedrivePolicy**

   - `Description`: Specifies the configuration for a dead-letter queue (DLQ), which stores messages that couldn’t be successfully processed after multiple attempts.
   - `Components`:
     - **`deadLetterTargetArn`**: The ARN of the DLQ.
     - **`maxReceiveCount`**: The maximum number of receive attempts for a message before moving it to the DLQ.
   - `Usage`: Configure this to handle unprocessable messages by moving them to a DLQ after a specified number of failed processing attempts.
   - `Impact`: Messages that exceed the `maxReceiveCount` are automatically sent to the DLQ, where they can be investigated without affecting the main queue’s performance.

7. **maxReceiveCount** (within `RedrivePolicy`)

   - `Description`: Specifies the maximum number of times a message can be received from the queue before it is moved to the DLQ.
   - `Range`: Must be set to 1 or greater.
   - `Usage`: Set `maxReceiveCount` based on the acceptable number of retry attempts. For instance, if processing is expected to succeed within 3 attempts, set `maxReceiveCount` to 3.
   - `Impact`: A higher value allows for more retries before sending the message to the DLQ, which is helpful in scenarios where transient errors are common. Lower values reduce the number of retries, sending unprocessable messages to the DLQ sooner.


In [ ]:
QUE_NAME = 'httx_sqs'

attributes = {
    # Maximum message size (default 262144 bytes or 256 KB, max 262144)
    'MaximumMessageSize': '262144',
    
    # Message retention period (default 345600 seconds or 4 days, max 1209600)
    'MessageRetentionPeriod': '86400',  # 1 day
    
    # Visibility timeout (default 30 seconds, max 43200 or 12 hours)
    'VisibilityTimeout': '60',  # 1 minute
    
    # Delivery delay (default 0 seconds, max 900 seconds or 15 minutes)
    'DelaySeconds': '5',  # 5 seconds
    
    # Enable content-based deduplication for FIFO queues only
    # Uncomment if using a FIFO queue
    # 'ContentBasedDeduplication': 'true',
    
    # Set queue type to FIFO (requires .fifo suffix for QueueName)
    # Uncomment if using a FIFO queue
    # 'FifoQueue': 'true',

    # Enable server-side encryption with KMS
    'KmsMasterKeyId': 'alias/aws/sqs',  # Use AWS managed key, or specify your own KMS Key ARN
    'KmsDataKeyReusePeriodSeconds': '300',  # 5 minutes reuse period for data encryption keys
    
    # Configure dead-letter queue (DLQ) if needed
    # Replace 'DeadLetterQueueARN' with your DLQ ARN
    # Uncomment and specify the DLQ ARN to use this option
    # 'RedrivePolicy': '{"maxReceiveCount":"5", "deadLetterTargetArn":"DeadLetterQueueARN"}'
}


In [ ]:
# Create the SQS queue with the specified attributes
que_url = sqs_client.create_queue(
    QueueName=QUE_NAME,
    Attributes=attributes
)

In [ ]:
# que_url = sqs_client.get_queue_url(QueueName=QUE_NAME)['QueueUrl']

In [ ]:
message_body = 'This is a SQS test message'
message_attributes = {}
response = sqs_client.send_message(
            QueueUrl=que_url,
            MessageBody=message_body,
            MessageAttributes=message_attributes or {}
        )
message_id = response['MessageId']

In [ ]:
response = sqs_client.receive_message(
            QueueUrl=que_url,
            MaxNumberOfMessages=10,
            WaitTimeSeconds=10
        )
messages = response.get('Messages', [])

len(messages)

In [ ]:
# Delete messages
for message in messages:
    print(message['Body'])
    # sqs_client.delete_message(
    #     QueueUrl=que_url,
    #     ReceiptHandle=message['ReceiptHandle']
    # )

# KMS (Key Management Service)

-   [AWS KMS - Key Management Service (Crash Course)](https://www.youtube.com/watch?v=f3APF1dP8w0)
-   [How to encrypt/decrypt data with AWS KMS and OpenSSL?](https://www.youtube.com/watch?v=w_rpeiG7xjQ)

AWS Key Management Service (KMS) is a fully managed service that enables you to create, control, and use encryption keys to protect your data across AWS services and applications. It integrates with many AWS services for encryption, such as S3, RDS, EBS, SQS, and Lambda, and provides a centralized solution to manage encryption keys.


1. **Customer Master Key (CMK)**
   - **Description**: A CMK is the primary key resource in KMS and can be used to encrypt, decrypt, and generate data keys. CMKs are unique to your AWS account and region.
   - **Types of CMKs**:
     - **AWS Managed CMK**: Managed by AWS, used by default in many AWS services (like S3, EBS). You don’t directly manage these keys.
     - **Customer Managed CMK**: Created and managed by you, giving you full control over the key’s lifecycle, including key rotation, deletion, and permissions.
     - **AWS Owned CMK**: Fully managed by AWS, shared across multiple accounts, and used for encryption at scale without user control.
   - **Key ID and Key ARN**: Each CMK has a unique Key ID and Amazon Resource Name (ARN) used for referencing the key in applications and services.
   
   **Example**: If you create a CMK with the alias `MyApplicationCMK`, you can use this key in S3 to encrypt and decrypt objects.

2. **Data Key**
   - **Description**: Data keys are encryption keys generated by KMS, specifically used for encrypting large data. Since CMKs are not intended to encrypt large amounts of data, data keys serve as the intermediaries.
   - **Types**:
     - **Plaintext Data Key**: A readable format key used directly in encryption.
     - **Encrypted Data Key**: An encrypted version of the plaintext key, encrypted with a CMK.
   - **Usage**: You generate data keys using a CMK, then encrypt data with the plaintext key and store the encrypted data key alongside the data. To decrypt, KMS decrypts the encrypted data key using the CMK, and the plaintext data key decrypts the data.

   **Example**: You generate a data key to encrypt a sensitive document stored in S3. Only the encrypted data key is saved with the document; the plaintext data key is discarded after encryption.

3. **Envelope Encryption**
   - **Description**: This is an encryption technique where data is encrypted using a data key, and that data key is encrypted with a CMK. It’s commonly used for efficient encryption in scenarios with large datasets.
   - **Process**:
     1. KMS generates a data key (plaintext and encrypted).
     2. Use the plaintext data key to encrypt your data.
     3. Store the encrypted data key with the encrypted data.
   - **Benefits**: Reduces reliance on CMKs to directly encrypt data, limits exposure of the plaintext key, and enables scalable encryption of large datasets.

   **Example**: A Lambda function encrypts sensitive user information before storing it in DynamoDB. The data key, encrypted by a CMK, is stored with the encrypted data in DynamoDB.

4. **Key Policies**
   - **Description**: Key policies define permissions on a CMK and specify which AWS users, roles, or services can access or manage a CMK.
   - **Structure**:
     - Managed in JSON format.
     - You can define actions (like `kms:Encrypt`, `kms:Decrypt`) and conditions based on AWS identity.
   - **Usage**: Key policies are the primary way to control access to CMKs, and they can be combined with IAM policies for finer control.

   **Example**: A CMK key policy allows the IAM role of an application to use the key for decryption while preventing other users from accessing it.

5. **Grants**
   - **Description**: A grant allows temporary, specific permissions on a CMK for AWS principals (users, roles, or services) without altering the key policy.
   - **Common Usage**: Used for temporary access or delegated access in specific applications. Grants specify the actions, such as `Encrypt` or `Decrypt`, that the principal is allowed to perform.
   - **Example**: A KMS grant might allow a specific Lambda function to encrypt data with a CMK for a limited time, without changing the key policy.

6. **Automatic Key Rotation**
   - **Description**: Key rotation automatically generates new cryptographic material (a new version) for a CMK at a regular interval (every year for AWS managed keys).
   - **Usage**: Rotation applies only to customer-managed CMKs and helps improve security by periodically refreshing keys. Existing data encrypted with older key material is still decryptable, as KMS keeps older versions of CMKs active for decryption.
   - **Example**: You enable automatic rotation for a CMK used to encrypt S3 objects, ensuring regular updating of cryptographic material without manual intervention.

7. **Custom Key Store**
   - **Description**: A custom key store allows you to use your own key management hardware, hosted within AWS CloudHSM, to generate and store keys, while integrating with KMS for ease of management.
   - **Use Case**: If you have strict compliance or regulatory requirements that require control over key generation and storage, custom key stores provide this by utilizing dedicated HSMs.
   - **Example**: An organization needing FIPS-compliant hardware uses a custom key store to generate CMKs on dedicated hardware while managing them in KMS.

8. **Multi-Region Keys**
   - **Description**: Multi-Region keys are a feature of KMS that allow you to replicate encryption keys across multiple AWS regions, enabling data encryption and decryption across regions.
   - **Benefits**: Facilitate multi-region disaster recovery, high availability, and data residency requirements.
   - **Example**: If an application encrypts data in multiple regions, multi-region keys provide consistent access across regions without manual key duplication.

##### **Example 1**: Encrypting Data in S3 Using Customer Managed CMK

In [ ]:
import boto3

# Initialize KMS client
kms_client = boto3.client('kms')

# Create a customer-managed CMK
response = kms_client.create_key(
    Description='Key for encrypting sensitive S3 data',
    KeyUsage='ENCRYPT_DECRYPT',
    Origin='AWS_KMS'
)
cmk_id = response['KeyMetadata']['KeyId']

# Use CMK to encrypt data
s3_client = boto3.client('s3')
bucket_name = 'my-secure-bucket'
object_key = 'sensitive-data.txt'
s3_client.put_object(
    Bucket=bucket_name,
    Key=object_key,
    Body='Sensitive information goes here',
    ServerSideEncryption='aws:kms',
    SSEKMSKeyId=cmk_id
)
print("Data encrypted and stored in S3.")

##### **Example 2**: Envelope Encryption for Large Data

In [ ]:
# Generate a data key using the CMK
data_key_response = kms_client.generate_data_key(KeyId=cmk_id, KeySpec='AES_256')
plaintext_data_key = data_key_response['Plaintext']
encrypted_data_key = data_key_response['CiphertextBlob']

# Encrypt data with the plaintext data key
from cryptography.fernet import Fernet
cipher = Fernet(plaintext_data_key)
encrypted_data = cipher.encrypt(b'Large data to encrypt')

# Store encrypted data and encrypted data key
store_data(encrypted_data, encrypted_data_key)
print("Data and data key encrypted and stored.")

# [SMS (Secret Management Service)](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/secretsmanager.html)

In [ ]:
# Initialize the Secrets Manager client
sms_client = boto3.client('secretsmanager', region_name='us-east-1')  # Replace 'us-east-1' with your AWS region


# Define secret name and value
secret_name = "prod/db_password"
secret_value = '{"username": "admin", "password": "mysecretpassword"}'

In [ ]:
response = sms_client.create_secret(
    Name=secret_name,
    Description="This is a sample secret for database credentials",
    SecretString=secret_value
)

In [ ]:
# Retrieve the secret value
response = sms_client.get_secret_value(SecretId=secret_name)

# The secret can be either a JSON string or a plain text value
if 'SecretString' in response:
    secret = response['SecretString']
    # If it's JSON formatted, load it into a Python dictionary
    secret =  json.loads(secret)
elif 'SecretBinary' in response:
    # Decode binary secrets
    secret = response['SecretBinary']


In [ ]:
response = sms_client.delete_secret(
    SecretId='string',
    # RecoveryWindowInDays=123,
    ForceDeleteWithoutRecovery=True # You can’t use both this parameter and RecoveryWindowInDays
)